### Import libraries

In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pandas as pd
from glob import glob
import scipy as sp

### Import data

Coordinate data is structured in tabular format. Columns represent sample ID, track ID, coordinates in x, y, and z, and time. Each row is a single time point.

In [2]:
# locate the input file
data_path = "example_data.xlsx"
files = sorted(glob(data_path))

print(*files, sep="\n")

example_data.xlsx


In [3]:
# create dataframe
df = pd.read_excel(files[0])
df.head()

,Embryo,Track,X_um,Y_um,Z_um,Time_s
0,1,1000000017,73.00,51.39,3.93,7
1,1,1000000017,75.91,52.55,4.40,8
2,1,1000000017,77.75,52.57,4.40,9
3,1,1000000017,78.25,52.47,3.77,10
4,1,1000000017,78.63,52.51,4.51,11


### Measure speed, acceleration, jerk, and snap per track

In [4]:
# initialise output dataframe
out_df = pd.DataFrame(columns = ['Embryo', 'Track', 'Velocity', 'Acceleration', 'Jerk', 'Snap'])

# find all track IDs
trackIDs = df.Track.unique()

# for every track:
for track in range(len(trackIDs)):

    current_track = df.loc[df.Track == trackIDs[track]]
    current_track = current_track.iloc[::2] # drop every other row (reduce time resolution)

    # save information about sample and track ID
    out_df.at[track, 'Embryo'] = current_track.iloc[0,0]
    out_df.at[track, 'Track'] = trackIDs[track]

    # only analyse if track is long enough
    if len(current_track) > 5:
        
        # velocity is in um/min (temporal resolution = 1 frame/min)
        current_velocity = []
        for frame in range(len(current_track)-1):
            pA = current_track.iloc[frame,2:5].to_numpy()
            pB = current_track.iloc[frame+1,2:5].to_numpy()
            tA = current_track.iloc[frame,5]
            tB = current_track.iloc[frame+1,5]
            with np.errstate(divide='ignore'):
                res = np.linalg.norm(pA - pB)/(tB - tA)
            # res[np.isnan(res)] = -2
            current_velocity = np.append(current_velocity, np.abs(res))

        # acceleration is in um/min2 (temporal resolution = 1 frame/min)
        current_acceleration = []
        for frame in range(len(current_velocity)-1):
            pA = current_velocity[frame]
            pB = current_velocity[frame+1]
            tA = current_track.iloc[frame,5]
            tB = current_track.iloc[frame+1,5]
            with np.errstate(divide='ignore'):
                res = (pB - pA)/(tB - tA)
            current_acceleration = np.append(current_acceleration, np.abs(res))
            
        # jerk is in um/min3 (temporal resolution = 1 frame/min)
        current_jerk = []
        for frame in range(len(current_acceleration)-1):
            pA = current_acceleration[frame]
            pB = current_acceleration[frame+1]
            tA = current_track.iloc[frame,5]
            tB = current_track.iloc[frame+1,5]
            current_jerk = np.append(current_jerk, np.abs((pB - pA)/(tB - tA)))
            
        # snap is in um/min4 (temporal resolution = 1 frame/min)
        current_snap = []
        for frame in range(len(current_jerk)-1):
            pA = current_jerk[frame]
            pB = current_jerk[frame+1]
            tA = current_track.iloc[frame,5]
            tB = current_track.iloc[frame+1,5]
            current_snap = np.append(current_snap, np.abs((pB - pA)/(tB - tA)))
            
        out_df.at[track, 'Velocity'] = np.mean(current_velocity)    
        out_df.at[track, 'Acceleration'] = np.mean(current_acceleration)    
        out_df.at[track, 'Jerk'] = np.mean(current_jerk)    
        out_df.at[track, 'Snap'] = np.mean(current_snap)

    # otherwise return NaN
    else:

        out_df.at[track, 'Velocity'] = np.nan  
        out_df.at[track, 'Acceleration'] = np.nan
        out_df.at[track, 'Jerk'] = np.nan
        out_df.at[track, 'Snap'] = np.nan

### Save output

In [5]:
out_df.to_csv("kinematics.csv")